### 🛠️ ধাপ ১: পরিবেশ পরীক্ষা (Environment Check)
এখানে আমরা চেক করছি পাইথন আমাদের **RTX A4500** জিপিউটি ডিটেক্ট করতে পারছে কি না।
- `torch.cuda.is_available()`: জিপিউ অ্যাক্টিভ আছে কি না তা জানায়।
- এটি নিশ্চিত করে যে মডেলটি CPU তে লোড হয়ে স্লো হবে না।

In [10]:

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

CUDA Available: True
GPU Name: NVIDIA RTX A4500
VRAM: 19.99 GB


#Eta korar por restart cell dite hoy

In [3]:
!pip install accelerate ipywidgets

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ---------------------------------------- 0.0/914.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/914.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/914.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/914.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/914.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/914.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/914.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/914.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/914.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/914.9 kB ? eta -:--:--
   ----------- ---------------------------- 262.1/914.9 kB ? eta -:--:--
   ----------- ---------------------------- 262.1/914.9 kB ? eta -:--:--
   ----------- ---------------------------- 262.1/914.9 kB ? eta -:--:-


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


### 🧠 ধাপ ২: মডেল লোড করা (Standard Mode)
এখানে আমরা `Qwen2.5-7B` মডেলটি লোড করছি।
- **`torch_dtype=torch.bfloat16`**: Ampere আর্কিটেকচার (RTX 3000/A-series) এর জন্য এটি সেরা ফরম্যাট। এটি `float32` এর চেয়ে অর্ধেক মেমোরি নেয় কিন্তু একুরেসি ঠিক রাখে।
- **`device_map="auto"`**: মডেলের লেয়ারগুলো অটোমেটিক্যালি VRAM এ সাজিয়ে নেয়।
- **`trust_remote_code=True`**: Qwen এর কাস্টম আর্কিটেকচার চালানোর অনুমতি দেয়।
*(নোট: এই মোডে মডেলটি প্রায় ১৪-১৫ জিবি VRAM নেবে)*

In [2]:
model_name = "Qwen/Qwen2.5-7B-Instruct"

print("Loading Model... (This will download approx 15GB data for the first time)")

# ১. মডেল লোড করা
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,  # RTX A4500 এর জন্য সেরা ফরম্যাট (মেমোরি বাঁচায়, স্পিড বাড়ায়)
    device_map="auto",           # অটোমেটিক GPU-তে লেয়ারগুলো সাজিয়ে নেবে
    trust_remote_code=True
)

# ২. টোকেনাইজার লোড করা
tokenizer = AutoTokenizer.from_pretrained(model_name)

print("Model Loaded Successfully!")

Loading Model... (This will download approx 15GB data for the first time)


`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

e:\SmartSearchSystem\SmartSearchSystem\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\RUET\.cache\huggingface\hub\models--Qwen--Qwen2.5-7B-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Model Loaded Successfully!


### 💬 ধাপ ৩: চ্যাট ফাংশন তৈরি
Qwen মডেল সাধারণ টেক্সট প্রম্পটের চেয়ে চ্যাট ফরম্যাটে ভালো কাজ করে।
- **`apply_chat_template`**: আমাদের সাধারণ টেক্সটকে `<|im_start|>system...` ফরম্যাটে রূপান্তর করে যা মডেল বুঝতে পারে।
- **`max_new_tokens=512`**: মডেল সর্বোচ্চ কত বড় উত্তর দেবে তা নির্ধারণ করে।
- **`do_sample=True`**: রোবোটিক উত্তরের বদলে কিছুটা সৃজনশীল উত্তর জেনারেট করে।

In [3]:
def ask_qwen(prompt, system_message="You are a helpful assistant."):
    # ১. মেসেজ ফরম্যাট সাজানো
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
    ]

    # ২. চ্যাট টেমপ্লেট অ্যাপ্লাই করা (ফাইলের নির্দেশনা অনুযায়ী)
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    # ৩. ইনপুট এনকোড করে জিপিউতে পাঠানো
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    # ৪. রেসপন্স জেনারেট করা
    with torch.no_grad(): # মেমোরি বাঁচাতে গ্রেডিয়েন্ট ক্যালকুলেশন বন্ধ
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=512,  # সর্বোচ্চ কত শব্দ বলবে
            temperature=0.7,     # সৃজনশীলতা নিয়ন্ত্রণ
            do_sample=True,
            top_p=0.9
        )

    # ৫. শুধু নতুন জেনারেট করা অংশটুকু ডিকোড করা
    # ইনপুট অংশটুকু বাদ দিয়ে শুধু উত্তরটা নেওয়া হচ্ছে
    output_ids = generated_ids[0][len(model_inputs.input_ids[0]):]
    response = tokenizer.decode(output_ids, skip_special_tokens=True)

    return response

In [6]:
# সাধারণ প্রশ্ন
response = ask_qwen("Explain Quantum Computing in simple terms.")
print("-" * 50)
print("Qwen Response:\n", response)
print("-" * 50)

# কোডিং প্রশ্ন (Qwen কোডিংয়ে খুব ভালো)
code_response = ask_qwen("Write a Python code function to check if a number is prime.")
print("Qwen Code:\n", code_response)

--------------------------------------------------
Qwen Response:
 Quantum computing is a type of computing that uses quantum-mechanical phenomena, such as superposition and entanglement, to perform operations on data. Here’s a simple way to understand it:

1. **Classical Bits vs. Quantum Bits (Qubits):**
   - In classical computers, information is processed using bits, which can be either 0 or 1.
   - In quantum computers, information is processed using qubits, which can be both 0 and 1 at the same time thanks to a property called superposition.

2. **Superposition:**
   - Imagine a coin spinning in the air; it's not heads or tails until it lands. A qubit acts similarly—it can be in multiple states simultaneously.

3. **Entanglement:**
   - Entanglement is like having two coins that are magically linked. If you flip one, the other instantly reflects the change, no matter how far apart they are. In quantum computing, qubits can become entangled, meaning the state of one (whether it's 0

In [7]:
if torch.cuda.is_available():
    allocated = torch.cuda.memory_allocated(0) / 1024**3
    cached = torch.cuda.memory_reserved(0) / 1024**3
    print(f"Currently Allocated VRAM: {allocated:.2f} GB")
    print(f"Total Reserved VRAM: {cached:.2f} GB")

Currently Allocated VRAM: 14.20 GB
Total Reserved VRAM: 14.28 GB


In [8]:
# এই ফাংশনটি মডেলকে নির্দিষ্ট কাজের জন্য প্রস্তুত করে (ফাইলের নির্দেশনা অনুযায়ী)
def create_coding_prompt(task, language="Python"):
    system_prompt = f"""<|im_start|>system
You are Qwen-Coder, an expert programming assistant. Generate clean, efficient, and well-documented code.
Requirements:
- Use {language} programming language
- Include comprehensive docstrings
- Add type hints where appropriate
<|im_end|>
<|im_start|>user
Task: {task}
Please provide a complete, production-ready solution.
<|im_end|>
<|im_start|>assistant"""
    return system_prompt

# ব্যবহারের উদাহরণ
task_description = "Create a class that manages a simple in-memory cache with TTL (time-to-live) support."
full_prompt = create_coding_prompt(task_description)

# টোকেনাইজ করা (সরাসরি টেক্সট ইনপুট, কারণ আমরা ম্যানুয়ালি ফরম্যাট করেছি)
model_inputs = tokenizer([full_prompt], return_tensors="pt").to(model.device)

# জেনারেশন
generated_ids = model.generate(**model_inputs, max_new_tokens=512, temperature=0.2) # কোডিংয়ের জন্য temperature কম রাখা ভালো
response = tokenizer.decode(generated_ids[0][len(model_inputs.input_ids[0]):], skip_special_tokens=True)

print("Expert Code Solution:\n")
print(response)

Expert Code Solution:


Certainly! Below is a Python class `SimpleCache` that manages an in-memory cache with Time-To-Live (TTL) support. The class uses a dictionary to store the key-value pairs and their expiration times.

```python
from typing import Any, Dict, Tuple
import time

class SimpleCache:
    """
    A simple in-memory cache with TTL support.

    Attributes:
        cache (Dict[str, Tuple[Any, float]]): Stores the key-value pairs and their expiration times.
    """

    def __init__(self):
        """
        Initializes the SimpleCache instance.
        """
        self.cache: Dict[str, Tuple[Any, float]] = {}

    def set(self, key: str, value: Any, ttl: int) -> None:
        """
        Sets a key-value pair in the cache with a specified TTL.

        Args:
            key (str): The key for the cache entry.
            value (Any): The value to be stored.
            ttl (int): The time-to-live in seconds for the cache entry.

        Returns:
            None
        

In [9]:
# একসাথে ৩টি প্রশ্ন
prompts = [
    "What is AI?",
    "What is Machine Learning?",
    "What is Deep Learning?"
]

# ফরম্যাট করা
formatted_prompts = [
    tokenizer.apply_chat_template([{"role": "user", "content": p}], tokenize=False, add_generation_prompt=True)
    for p in prompts
]

# প্যাডিং সহ টোকেনাইজ করা (ব্যাচ প্রসেসিংয়ে সাইজ সমান হতে হয়)
inputs = tokenizer(formatted_prompts, return_tensors="pt", padding=True, truncation=True).to(model.device)

# জেনারেশন
generated_ids = model.generate(**inputs, max_new_tokens=100)

# ডিকোড করা
responses = tokenizer.batch_decode(generated_ids[:, inputs.input_ids.shape[1]:], skip_special_tokens=True)

for q, a in zip(prompts, responses):
    print(f"Question: {q}")
    print(f"Answer: {a}\n")

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Question: What is AI?
Answer: - **Artificial Intelligence (AI)** refers to the simulation of human intelligence in machines that are programmed to think and learn like humans. This can include tasks such as visual perception, speech recognition, decision-making, and translation between languages.

Key aspects of AI include:

1. **Machine Learning**: A subset of AI where systems can automatically learn and improve from experience without being explicitly programmed. It involves algorithms that can access data and use it to learn for themselves.

2. **Deep Learning**: A type

Question: What is Machine Learning?
Answer: Machine learning is a subset of artificial intelligence (AI) that involves the development of algorithms and statistical models that enable computer systems to improve their performance on a specific task through experience or data exposure without being explicitly programmed. The goal of machine learning is to allow computers to learn from and make decisions or prediction